In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI environment. env.P represents the transition probabilities of the environment.
        theta: Stopping threshold. If the value of all states changes less than theta
            in one iteration we are done.
        discount_factor: lambda time discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    
    # Implement!    
    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    
    while True:
        V_new = np.zeros(env.nS)

        for s in range(env.nS):
            max_action_value = np.NINF
            for a in env.P[s]:
                action_value = 0
                for transition_probability, next_state, immediate_reward, done in env.P[s][a]:
                    action_value += immediate_reward + discount_factor * V[next_state]
                if action_value > max_action_value:
                    max_action_value = action_value
                    
            V_new[s] = max_action_value
            
        max_change = np.max(abs(V_new - V))
            
        if max_change < theta:
            break
        else:
            V = V_new

    for s in range(env.nS):
        max_state_value = np.NINF
        max_state = []
        for a in env.P[s]:
            for transition_probability, next_state, immediate_reward, done in env.P[s][a]:
                if a not in max_state and V[next_state] >= max_state_value:
                    if V[next_state] > max_state_value:
                        max_state.clear()
                        max_state_value = V[next_state]
                    max_state.append(a)
                    
        new_action_probability = float(1) / len(max_state)
        
        for next_s in range(len(policy[s])):
            if next_s in max_state:
                policy[s][next_s] = new_action_probability
            else:
                policy[s][next_s] = 0

    return policy, V

In [4]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[ 0.25  0.25  0.25  0.25]
 [ 0.    0.    0.    1.  ]
 [ 0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5 ]
 [ 1.    0.    0.    0.  ]
 [ 0.5   0.    0.    0.5 ]
 [ 0.25  0.25  0.25  0.25]
 [ 0.    0.    1.    0.  ]
 [ 1.    0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25]
 [ 0.    0.5   0.5   0.  ]
 [ 0.    0.    1.    0.  ]
 [ 0.5   0.5   0.    0.  ]
 [ 0.    1.    0.    0.  ]
 [ 0.    1.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [5]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)